In [0]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [10]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix


In [11]:
ls data

men_shoes.csv


In [14]:
df = pd.read_csv('data/men_shoes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
def run_model_feats(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [21]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model_feats(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [22]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model_feats(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [25]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [36]:
dict_str = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(dict_str)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [37]:
literal_eval(dict_str)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':return output_dict
  features = literal_eval(x.replace('\\"', '"')) # gdy zamienia na puste  pojawia się problem z nieparzystością cudzysłowów bo: "17"\\"" zamienia na "17"" (trzy cudzusłowy) 
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  return output_dict

In [0]:
df['parsed_features'] = df['features'].map(parse_features)

In [67]:
df['parsed_features'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [71]:
keys = set()

df['parsed_features'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [77]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['parsed_features'].map(lambda feats: feats[key] if key in feats else np.nan )

In [86]:
keys_stat = {}

for key in tqdm_notebook(keys):
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [94]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [104]:
df.brand = df.brand.map(lambda x: str(x).lower())
df[df.brand == df.feat_brand][['brand', 'feat_brand']].shape[0]

8846

In [274]:
feats_cat = [x for x in df.columns if '_cat' in x and x !='feat_catalog']
df[feats_cat].sample()

,brand_cat,feat_brand_cat,feat_color_cat,feat_gender_cat,feat_manufacturer part number_cat,feat_material_cat,feat_sport_cat,feat_style_cat,feat_includes_cat,feat_autographed_cat,feat_color/finish family_cat,feat_height_cat,feat_number of pockets_cat,feat_necklace type_cat,feat_country of manufacture_cat,feat_pant style_cat,feat_batteries included?_cat,feat_mpn#_cat,feat_100% adidas authentic_cat,feat_has expiration_cat,feat_country of manufacturer_cat,feat_band material_cat,feat_fishing_cat,feat_dial markers_cat,feat_diamond clarity_cat,feat_waist size_cat,feat_smart watch_cat,feat_picture_cat,feat_is waterproof_cat,feat_sub type_cat,feat_item style_cat,feat_box_cat,feat_golf shoe type_cat,feat_item type_cat,feat_case diameter_cat,feat_place of origin_cat,feat_fabrication_cat,feat_catalog_cat,feat_ring size_cat,feat_package_cat,...,feat_looks best on these face shapes:_cat,feat_compass_cat,feat_clothing type_cat,feat_kids backpacks_cat,feat_product #_cat,feat_navy blue_cat,feat_quantity in set_cat,feat_is lined_cat,feat_fits models_cat,feat_ring style_cat,feat_date_cat,feat_accessory type_cat,feat_vendor description_cat,feat_fabric care instructions_cat,feat_era_cat,feat_ean_cat,feat_antiscratch lens coating_cat,feat_fit:_cat,feat_shoe height_cat,feat_fuel type_cat,feat_weight_cat,feat_fabric content_cat,feat_pocket_cat,feat_shoe size_cat,feat_hat style_cat,feat_has mercury_cat,feat_reinforced knee_cat,feat_wheel type_cat,feat_made in_cat,feat_boxed-product dimensions_cat,feat_mechanic_cat,feat_temple length_cat,feat_hunting_cat,"feat_thick, warm, cozy_cat",feat_adjustable_cat,feat_heart rate monitor_cat,feat_profession_cat,feat_number of pieces_cat,feat_heel height_cat,feat_model number_cat
3168,619,305,3,0,540,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,26,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1


In [0]:
{feat: df[feat].unique() for feat in df[feats_cat]}

In [0]:
df['weight_g_cat'] = df['weight_g'].factorize()[0]
df

In [0]:
feats += feats_cat
feats = list(set(feats))

In [121]:
df['brand'].value_counts(normalize = True)

nike                                0.097210
puma                                0.033315
ralph lauren                        0.028775
vans                                0.021116
new balance                         0.020295
                                      ...   
onitsuka tiger by asics             0.000055
adidas crazy1 kobe bb shoes         0.000055
red wing heritage                   0.000055
luos cultural goods (usa seller)    0.000055
holiday                             0.000055
Name: brand, Length: 1732, dtype: float64

In [134]:
df[df['brand'] == 'nike'].parsed_features.sample(5).values

array([{}, {},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '10.5', 'size': '10.5', 'color': 'red', 'model': '631047 601', 'manufacturer part number': '631047 601', 'brand': 'nike', 'age group': 'adult'},
       {'sport': 'football', 'condition': 'new with box'},
       {'sport': 'football', 'type': 'cleats', 'condition': 'new with box'}],
      dtype=object)

In [0]:
df['weight_g'] = df['weight'].map(lambda x: unit_converter(str(x).lower().strip()))

In [0]:
def unit_converter(x):
  units = {
      'lbs': 453.59237,
      'lb': 453.59237,
      'g': 1,
      'pounds': 453.59237,
      'ounces': 28.34952,
      'kg': 1000,
  }
  if x is None: return 0
  if str(x) == 'nan': return 0
  return float(x[:x.find(' ')]) * float(units.get(x[x.find(' '):].strip()))


In [272]:
df['weight_g'].unique()

array([0.00000000e+00, 1.36077711e+03, 9.00000000e+00, 6.57708937e+02,
       2.04116567e+02, 4.53592370e+02, 1.04326245e+02, 2.26796185e+03,
       2.49475804e+03, 3.37926316e+03, 1.81436948e+03, 1.26865250e+03,
       1.76901024e+03, 2.08652490e+03, 9.52543977e+02, 5.01537084e+02,
       6.80388555e+03, 6.80388480e+01, 4.54000000e+02, 4.76271988e+01,
       2.57980632e+02, 2.17724338e+03, 2.76691346e+03, 2.94835041e+03,
       5.00811336e+02, 1.30000000e+03, 9.10000000e+01, 9.07184740e+03,
       2.72155422e+03, 3.86000000e+02, 3.67409820e+02, 2.04116566e+03,
       1.41747600e+01, 9.07184740e+02, 1.41974412e+03, 2.67619498e+03,
       2.78959308e+03, 8.84505121e+02, 9.75223595e+02, 1.40000000e+04,
       2.17180027e+02, 4.53592370e+03, 1.72365101e+02, 1.13398093e+03,
       3.12579574e+04, 4.50000000e+01, 5.93752412e+03, 9.52543977e+01,
       7.59767220e+03, 2.85763193e+03, 2.72000000e+02, 1.80000000e+03,
       1.27005864e+03, 4.53592370e+01, 2.29064147e+03, 1.27005864e+02,
      

In [260]:
waga = '1.4 kg'
waga.find(' ')
value = float(waga[:waga.find(' ')])
value * 10 

14.0

In [261]:
  units = {
      'lbs': 453.59237,
      'g': 1,
      'pounds': 453.59237,
      'ounces': 28.34952,
      'kg': 1000,
  }
  unit = waga[waga.find(' '):].strip()
  factor = units.get(unit)
  factor *10

10000

In [0]:
feats = ['brand_cat', 
         'feat_brand_cat', 
         'feat_color_cat', 
         'feat_gender_cat', 
         'feat_material_cat', 
         'feat_sport_cat', 
         'feat_style_cat',
         'feat_weight_cat',
         'weight_g'
         ]

In [287]:

model = RandomForestRegressor(max_depth=100, n_estimators=100)
run_model_feats(feats, model)

(-45.88772199198429, 4.3446661918219815)

In [189]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X, y)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)


Weight,Feature
0.2507 ± 0.0166,brand_cat
0.1105 ± 0.0059,feat_material_cat
0.0129 ± 0.0010,feat_weight_cat
0.0119 ± 0.0015,feat_brand_cat
0.0097 ± 0.0012,feat_fabric content_cat
0.0077 ± 0.0013,feat_adjustable_cat
0.0076 ± 0.0026,feat_resizable_cat
0.0060 ± 0.0004,feat_shoe category_cat
0.0048 ± 0.0005,feat_fabric material_cat
0.0036 ± 0.0005,feat_jacket length_cat


In [292]:
!git add day5.ipynb

fatal: pathspec 'day5.ipynb' did not match any files
